In [1]:
import os
import pandas as pd
import geopandas as gpd
import geoparquet as gpq
import duckdb
from utils import *

In [2]:
def export_geoparquet(geo_package, output_path, filename, id = 'ID'):
    geo_package.to_crs("EPSG:4326", inplace=True)
    geo_package['wkt'] = geo_package['geometry'].to_wkt()
    
    geo_dataframe = gpd.GeoDataFrame()
    geo_dataframe["geometry"] = gpd.GeoSeries.from_wkt(geo_package["wkt"])    
    geo_dataframe = gpd.GeoDataFrame(geo_dataframe, geometry="geometry")
    geo_dataframe[id] = geo_package[id]

    geo_parquet = geo_dataframe.merge(geo_package[[i for i in geo_package.columns if i not in ["geometry", "wkt"]]],
                                    left_on=id, right_on = id)

    geo_parquet = geo_parquet.dropna(subset=['geometry'])
    if output_path is not None and filename is not None:
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        geo_parquet.fillna(0, inplace=True)
        geo_parquet.to_parquet(f'{output_path}/{filename}', compression = 'zstd')

ACCESSIBILITY_METRICS_DIR = OUTPUT_DIR + "/accessibility_metrics"

lba = LOCAL_BOARD_AREAS.copy()

OPTION_COMPARISON_PATH = '/Users/tszchun.chow/Library/CloudStorage/OneDrive-SharedLibraries-AucklandTransport/Congestion Charging - General/Lead Advisor (shared folder)/02 Analysis (Transport Modelling)/Stage A Options Assessment/StageA_Options_Comparison_FINAL.xlsx'
# trips_by_lba = gpd.GeoDataFrame(pd.read_excel(OPTION_COMPARISON_PATH, sheet_name='charged_pop')# .merge(lba[['LBA_2', 'geometry']], 
                                                                      # left_on='lba', 
                                                                      # right_on='LBA_2',
                                                                      # how = 'left'
                                                                      # )
#                     ,geometry='geometry', crs='epsg:2193')
trips_by_lba = pd.read_excel(OPTION_COMPARISON_PATH, sheet_name='charged_pop')
trips_by_lba.columns = ['scenario','lba','scenario_lba','veh_trips','change_veh_trips','%_change_veh_trips','charged_vehs', '%_charged', 'pt_trips','change_pt_trips','%_change_pt_trips']
trips_by_lba['scenario'] = "Option " + trips_by_lba['scenario']
trips_by_lba.to_parquet(os.path.join(ACCESSIBILITY_METRICS_DIR, "trips_by_LBA.parquet"))
# export_geoparquet(trips_by_lba, ACCESSIBILITY_METRICS_DIR, "trips_by_LBA.parquet", id="lba")

In [18]:
LBA_DICT = {'Aotea/Great Barrier': 0, 
    'Rodney': 1, 
 'Hibiscus and Bays': 2,
 'Upper Harbour': 3
,'Kaipatiki':4
,'Devonport - Takapuna':5
,'Waitakere Ranges':6
,'Henderson - Massey':7
,'Whau':8
,'Puketapapa':9
,'Albert - Eden':10
,'Waitemata':11
,'Orakei':12
,'Maungakiekie - Tamaki':13
,'Mangere - Otahuhu':14
,'Otara - Papatoetoe':15
,'Howick':16
,'Manurewa':17
,'Papakura':18
,'Franklin':19
,'External':20}

In [19]:
veh_pht = '../outputs/throughput_metrics/veh_pht_byLBA.csv'
veh_pht_df = pd.read_csv(veh_pht)[['origin_LBA', 'destination_LBA', 'variable', 'value']]# .to_parquet('../outputs/throughput_metrics/veh_pht_byLBA.parquet')
veh_pht_pivot = veh_pht_df.pivot(index=['origin_LBA', 'destination_LBA'], columns='variable', values='value').reset_index()
veh_pht_pivot['origin_index'] = veh_pht_pivot['origin_LBA'].map(LBA_DICT)
veh_pht_pivot['destination_index'] = veh_pht_pivot['destination_LBA'].map(LBA_DICT)
veh_pht_pivot.to_parquet('../outputs/throughput_metrics/veh_pht_byLBA.parquet')
veh_pht_pivot

variable,origin_LBA,destination_LBA,PHT_18112_Vehicle_All,PHT_18212_Vehicle_All,PHT_18312_Vehicle_All,PHT_26116_Vehicle_All,PHT_26150_Vehicle_All,PHT_26152_Vehicle_All,PHT_26154_Vehicle_All,PHT_26156_Vehicle_All,...,Trips_26366_Vehicle_All,Trips_26370_Vehicle_All,Trips_26372_Vehicle_All,Trips_26374_Vehicle_All,Trips_26376_Vehicle_All,Trips_26378_Vehicle_All,Trips_26380_Vehicle_All,Trips_26382_Vehicle_All,origin_index,destination_index
0,Albert - Eden,Albert - Eden,89286.359906,68553.163207,85676.627495,92189.302552,90924.491278,92070.368460,93155.463437,72049.696110,...,13175.354763,12370.914693,13213.412929,13280.557449,13114.132886,12694.407563,13130.947850,13136.372134,10,10
1,Albert - Eden,Aotea/Great Barrier,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10,0
2,Albert - Eden,Devonport - Takapuna,6410.083846,2789.749646,11088.431069,7600.334679,7389.737822,7366.063519,7727.991979,5823.271270,...,316.006936,389.809585,415.338058,311.215198,379.825380,331.126992,376.475923,387.545561,10,5
3,Albert - Eden,External,9502.135362,8625.499642,15093.181633,10777.462412,10780.983032,10902.542698,10913.871567,10640.941162,...,350.346864,350.401642,350.430949,350.376714,350.510159,350.564104,350.507541,350.487537,10,20
4,Albert - Eden,Franklin,7651.806536,6994.725153,20673.626024,9551.821854,9544.589669,9657.283466,9581.193097,9102.154793,...,336.159675,346.779027,339.446227,336.120478,346.724460,324.153402,346.839675,342.539896,10,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,Whau,Rodney,7910.338864,4795.974139,20381.402183,9595.851387,9622.253698,9621.979692,9674.954479,9768.974347,...,576.572680,580.304690,535.128858,577.095280,577.456820,594.994282,577.257041,577.864151,8,1
437,Whau,Upper Harbour,11384.664988,5320.418110,9759.489000,14917.625073,14796.548800,14830.780904,14989.134596,15195.393202,...,399.247048,408.102392,382.228783,399.106460,404.635034,418.280644,404.769244,404.975883,8,3
438,Whau,Waitakere Ranges,18641.229064,19708.909464,37749.257941,19197.235657,19143.193638,19051.777324,19135.436165,19196.659853,...,2647.576493,2642.529880,2636.268331,2643.048384,2638.771348,2652.550253,2638.124385,2639.299179,8,6
439,Whau,Waitemata,124954.621959,26719.493051,27389.520933,134384.300730,107803.488526,105563.746409,95805.770982,93041.180410,...,1452.561362,1683.161775,1756.097131,1447.684265,1740.004738,1581.041700,1742.070427,1742.539336,8,11


In [4]:
trips_by_lba[trips_by_lba['scenario']=='Option 1A']

,scenario,lba,scenario_lba,veh_trips,change_veh_trips,%_change_veh_trips,charged_vehs,pt_trips,change_pt_trips,%_change_pt_trips,LBA_2,geometry,area,wkt
0,Option 1A,PnR Zones,1A_PnR Zones,0.000000,0.000000,NaN,0.17900,NaN,0.000000,0.000000,NaN,None,NaN,None
1,Option 1A,Rodney,1A_Rodney,25461.293205,77.502306,0.003053,410.67270,0.016129,2106.561012,37.493765,Rodney,"MULTIPOLYGON (((175.08383 -35.69983, 175.09081...",5.809372e+09,"MULTIPOLYGON (((175.083831 -35.69983, 175.0908..."
2,Option 1A,Hibiscus and Bays,1A_Hibiscus and Bays,34425.244438,-52.634329,-0.001527,739.56800,0.021483,5255.254905,94.899356,Hibiscus and Bays,"MULTIPOLYGON (((174.69834 -36.54009, 174.69896...",4.546346e+08,"MULTIPOLYGON (((174.69834 -36.54009, 174.69895..."
3,Option 1A,Upper Harbour,1A_Upper Harbour,29697.771276,-29.634393,-0.000997,770.37130,0.025940,3998.682468,82.551251,Upper Harbour,"MULTIPOLYGON (((174.7247 -36.70573, 174.72492 ...",8.276195e+07,"MULTIPOLYGON (((174.7247 -36.70573, 174.724916..."
4,Option 1A,Kaipatiki,1A_Kaipatiki,27344.491132,-190.337008,-0.006913,978.47760,0.035783,6640.556319,277.299643,Kaipatiki,"MULTIPOLYGON (((174.73726 -36.76113, 174.73791...",4.917082e+07,"MULTIPOLYGON (((174.737263 -36.761127, 174.737..."
5,Option 1A,Devonport - Takapuna,1A_Devonport - Takapuna,17871.716397,-34.577657,-0.001931,630.60620,0.035285,4516.267672,117.813841,Devonport - Takapuna,"MULTIPOLYGON (((174.74165 -36.74717, 174.74212...",5.582238e+07,"MULTIPOLYGON (((174.741654 -36.747166, 174.742..."
6,Option 1A,Waitakere Ranges,1A_Waitakere Ranges,15634.113271,-10.411654,-0.000666,838.65000,0.053642,2958.253777,52.660188,Waitakere Ranges,"MULTIPOLYGON (((174.48148 -36.83341, 174.48174...",9.430090e+08,"MULTIPOLYGON (((174.481481 -36.83341, 174.4817..."
7,Option 1A,Henderson - Massey,1A_Henderson - Massey,36438.965439,-214.878336,-0.005862,1689.35600,0.046361,7336.451105,159.800466,Henderson - Massey,"MULTIPOLYGON (((174.59204 -36.7935, 174.59496 ...",6.461101e+07,"MULTIPOLYGON (((174.592043 -36.793503, 174.594..."
8,Option 1A,Whau,1A_Whau,27340.811919,-200.198408,-0.007269,1691.01600,0.061850,6855.429961,85.710486,Whau,"MULTIPOLYGON (((174.6943 -36.88604, 174.69962 ...",4.931230e+07,"MULTIPOLYGON (((174.694298 -36.886036, 174.699..."
9,Option 1A,Waitemata,1A_Waitemata,34965.921410,-1216.231750,-0.033614,1546.67600,0.044234,11575.130420,503.816019,Waitemata,"MULTIPOLYGON (((174.73558 -36.83248, 174.74074...",2.792792e+07,"MULTIPOLYGON (((174.73558 -36.832483, 174.7407..."
